In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv
import os

In [37]:
load_dotenv("../.env")

True

In [6]:
# tool create

@tool
def multiply(a:int, b:int) -> int:
    """
    Given: 2 numbers a and b
    Returns: product of a and b
    """
    return a * b

In [7]:
print(multiply.invoke({'a':3, 'b': 9}))

27


In [8]:
multiply.name, multiply.description, multiply.args

('multiply',
 'Given: 2 numbers a and b\nReturns: product of a and b',
 {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}})

In [14]:
# tool bind

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
llm_with_tools = llm.bind_tools([multiply])

In [ ]:
# List of tool suggested by LLM

llm_with_tools.invoke("What is 5 multiply by 10").tool_calls

[{'name': 'multiply',
  'args': {'a': 5.0, 'b': 10.0},
  'id': '4b3986e3-55b5-49c7-8d5f-79006ae061bc',
  'type': 'tool_call'}]

In [26]:
messages = []

query = HumanMessage("What is 5 multiply by 10")

messages.append(query)


In [28]:
messages

[HumanMessage(content='What is 5 multiply by 10', additional_kwargs={}, response_metadata={})]

In [30]:
result = llm_with_tools.invoke(messages)
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 5.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-8c5e8e39-dd41-4f17-aa78-90ca324763ff-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5.0, 'b': 10.0}, 'id': '3b41f5c3-ddd9-49b9-8356-b35d8d49a0f6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 33, 'output_tokens': 5, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}})

In [31]:
messages.append(result)

In [32]:
messages

[HumanMessage(content='What is 5 multiply by 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 5.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-8c5e8e39-dd41-4f17-aa78-90ca324763ff-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5.0, 'b': 10.0}, 'id': '3b41f5c3-ddd9-49b9-8356-b35d8d49a0f6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 33, 'output_tokens': 5, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}})]

In [24]:
multiply.invoke(result.tool_calls[0]['args'])

50

In [33]:
tool_message = multiply.invoke(result.tool_calls[0])
messages.append(tool_message)

In [34]:
messages

[HumanMessage(content='What is 5 multiply by 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 5.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-8c5e8e39-dd41-4f17-aa78-90ca324763ff-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5.0, 'b': 10.0}, 'id': '3b41f5c3-ddd9-49b9-8356-b35d8d49a0f6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 33, 'output_tokens': 5, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='50', name='multiply', tool_call_id='3b41f5c3-ddd9-49b9-8356-b35d8d49a0f6')]

In [36]:
llm_with_tools.invoke(messages).content

'The answer is 50.'

#### Currency Conversion Tool

In [ ]:
# tool create

from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency:str, target_currency:str) -> float:
    """
    Fetch the currency conversion factor between a base currency and target currency.
    """
    url = f"https://v6.exchangerate-api.com/v6/{os.getenv("EXCHANGE_RATE_API")}/pair/{base_currency}/{target_currency}"

    res = requests.get(url=url)

    return res.json()


In [127]:
@tool
def convert(base_currency_value: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    given a currency conversion rate, this function calculate a target currency value from a given base currency value
    """
    return base_currency_value * conversion_rate

In [128]:
# tool binding

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash')
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [129]:
messages = []

query = HumanMessage("What is the current exchange rate from USD to INR? Based on that, please convert 10 USD to INR.")

messages.append(query)

In [130]:
ai_message = llm_with_tools.invoke(messages)
messages.append(ai_message)
messages

[HumanMessage(content='What is the current exchange rate from USD to INR? Based on that, please convert 10 USD to INR.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4e7d8130-a11c-45e2-8332-20c124c317fb-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '4d7c0223-d388-4a53-b9ef-febd872d7b0d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 13, 'total_tokens': 104, 'input_token_details': {'cache_read': 0}})]

In [131]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '4d7c0223-d388-4a53-b9ef-febd872d7b0d',
  'type': 'tool_call'}]

In [132]:
import json

for tool_call in ai_message.tool_calls:
    # execute the 1st tool and get the conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        
        # fetch conversion rate
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        # append this message to messages
        messages.append(tool_message1)
    # execute the 2nd tool and get the coverted value from 1st tool
    if tool_call['name'] == 'convert':
        # fetch the current argument
        tool_call['args']['conversion_rate'] = conversion_rate

        # get the converted value
        tool_message2 = convert.invoke(tool_call)
        # append this message to messages
        messages.append(tool_message2)

        print(tool_message2)





In [133]:
messages

[HumanMessage(content='What is the current exchange rate from USD to INR? Based on that, please convert 10 USD to INR.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4e7d8130-a11c-45e2-8332-20c124c317fb-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '4d7c0223-d388-4a53-b9ef-febd872d7b0d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 13, 'total_tokens': 104, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1750204802, 